In [2]:
import sqlite3
import pandas as pd
import numpy as np
from typing import Dict, Any
from datetime import timedelta

In [3]:
def connect_to_db(db_path: str) -> sqlite3.Connection:
    """Efficiently connect to SQLite database"""
    return sqlite3.connect(db_path, detect_types=sqlite3.PARSE_DECLTYPES)

def preprocess_datetime(df: pd.DataFrame) -> pd.DataFrame:
    """Central datetime preprocessing to reduce redundant operations"""
    datetime_columns = ['创建时间', '预定日期', '下单时间']
    for col in datetime_columns:
        if col in df.columns:
            if not pd.api.types.is_datetime64_any_dtype(df[col]):
                try:
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                except Exception as e:
                    print(f"转换 {col} 列时出错：{e}")
    return df

In [17]:
def analyze(conn):
    """主分析函数"""
    catering_df = pd.read_sql_query("SELECT * FROM Catering", conn)
    
    results = marketing_analysis(catering_df)
    results1 = catering_df[catering_df['使用优惠'].notnull()]
    results_top = results1['使用优惠'].head(20)

    
    def convert_df_to_dict(data):
        if isinstance(data, pd.DataFrame):
            return data.to_dict(orient='records')
        return data

    processed_results = {}
    for category, data in results.items():
        processed_results[category] = {
            key: convert_df_to_dict(value) for key, value in data.items()
        }

    return results_top

In [5]:
def get_db_connection():
    conn = sqlite3.connect('db/ideapod.db')
    conn.row_factory = sqlite3.Row
    return conn


In [6]:
def marketing_analysis(catering_df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    """营销分析保持不变"""
    catering_df = preprocess_datetime(catering_df)
    catering_df['订单月份'] = catering_df['下单时间'].dt.to_period('M')
    catering_df['促销类型'] = catering_df['使用优惠'].fillna('无优惠')

    promotion_analysis = catering_df.groupby(['订单月份', '促销类型']).agg(
        订单总数=('订单号', 'count'),
        销售收入=('实收', 'sum'),
        平均折扣=('打折', 'mean'),
        总折扣金额=('打折', 'sum')
    ).reset_index()
    promotion_analysis['订单月份'] = promotion_analysis['订单月份'].astype(str)

    return {
        '促销优惠分析（数据有误）_bar': promotion_analysis
    }

In [18]:
print (analyze(get_db_connection()))

2              大众点评套餐
8               洛克85折
26             大众点评套餐
42             大众点评套餐
57               员工饮料
58                 赠送
68          众安最福利积分使用
73             大众点评套餐
81             场景套餐赠送
87              员工82折
94               员工饮料
101              员工饮料
124             洛克85折
136    员工饮料,员工饮料,员工饮料
138         众安最福利积分使用
144         众安最福利积分使用
145         众安最福利积分使用
150             洛克85折
154            大众点评套餐
157            大众点评套餐
Name: 使用优惠, dtype: object
